In [2]:
import itertools

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=4 #lattice dim
m=n+1 #space dim

R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
short = []
vshort = []
K = ceil(4*R)
K2 = (4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = []
for x in forb:
    for t in Permutations(list(x)):
        c = basis_coefs(t)
        if c not in forbd:
            forbd.append(c)
print("forbidden without symmetries:",len(forbd))
###


def find_smallest_mod(cc):
    A = set()
    maxv = 0
    for pt in forbd:
        p = abs(sum(cc[i]*pt[i] for i in range(n)))
        if p==0:
            return Infinity
        if p>maxv:
            maxv=p
        A.add(p)
    for i in range(1,maxv+2):
        found = True
        for k in range(1,floor(maxv/i)+1):
            if i*k in A:
                found = False
        if found:
            return i

        
#exhaustive verification up to K
K = 54
minv = Infinity
SS = set([i+1 for i in range(K)])
order = [(-1)^i*(1/4+i/2)-1/4 for i in range(K)]
C = []
for ccc in itertools.product(order,repeat=n):
    if ccc[0]>=0:
        C.append(list(ccc))
print(len(C))
for c in C:
    found = True
    S = SS.copy()
    for pt in forbd:
        p = abs(sum(c[i]*pt[i] for i in range(n)))
        if p==0:
            found = False
            break
        S -= {p}
        if not S:
            found = False
            break
    if found:
        v = find_smallest_mod(c)
        if v<minv:
            print(c,v)
            minv = v
        
print("done")        


very short: 4
short: 20
forbidden: 18
forbidden without symmetries: 370
4251528
[1, -3, 8, 15] 54
done
